In [1]:
## Get some random module
!pip install nbastats
import nbastats.nbastats as nbastats
!pip install py-goldsberry
import goldsberry

In [19]:
goldsberry.player.demographics(201566).player_info()[0]['HEIGHT']

u'6-3'

In [2]:
## Import a bunch of modules that may be helpful
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
import requests

In [326]:
## Saving what we merged, checking the shape to see how many shots we didn't have
joined_shots.to_csv("joined_shots_2014.csv")
print joined_shots.shape,all_shots.shape,all_chart.shape

(200532, 40) (203590, 21) (205550, 22)


In [9]:
## function for getting the shot log of a player given his ID and the year
def shot_log(year,player_id):
    api_param = {'DateFrom' : '',
                    'DateTo' : '',
                    'GameSegment': '',
                    'LastNGames': 0,
                    'LeagueID': "00",
                    'Location': '',
                    'Month': 0,
                    'OpponentTeamID': 0,
                    'Outcome': '',
                    'Period': 0,
                    'PlayerID': player_id,
                    'Season': year,
                    'SeasonSegment': '',
                    'SeasonType': "Regular Season",
                    'TeamID': 0,
                    'VsConference':'',
                    'VsDivision': ''
                               }
    pull=requests.get("http://stats.nba.com/stats/playerdashptshotlog?",params=api_param)
    headers = pull.json()['resultSets'][0]['headers']
    values = pull.json()['resultSets'][0]['rowSet']
    huh=pd.DataFrame([dict(zip(headers, value)) for value in values])
    return huh


In [4]:
# get players in season a
def players(a):
    url = "http://stats.nba.com/stats/commonallplayers?"
    api_param = {'IsOnlyCurrentSeason':"0",
                          'LeagueID': '00',
                          'Season': '2015-16'}
    pull = requests.get(url, params=api_param)
    _headers = pull.json()['resultSets'][0]['headers']
    _values = pull.json()['resultSets'][0]['rowSet']
    season=pd.DataFrame([dict(zip(_headers, value)) for value in _values])
    season = season.query("FROM_YEAR <= @a and TO_YEAR >= @a")
    return season

In [5]:
players2014=players('2014')
players2013=players('2013')

In [6]:
height_2013 = []
for indx,row in players2013.iterrows():
    height_2013.append(goldsberry.player.demographics(row['PERSON_ID']).player_info()[0]['HEIGHT'])

In [7]:
len(height_2013),len(players2013.index)

(505, 505)

In [8]:
players2013['HEIGHT'] = height_2013

In [9]:
height_2014 = []
for indx,row in players2014.iterrows():
    height_2014.append(goldsberry.player.demographics(row['PERSON_ID']).player_info()[0]['HEIGHT'])
players2014['HEIGHT'] = height_2014   

In [293]:
#Get shot chart and shot log for all players in 2014
#Don't run, already been saved
all_shots= pd.DataFrame()
all_chart = pd.DataFrame()
for index, row in players2014.iterrows():
    chart = nbastats.ShotChart(row["PERSON_ID"],season="2014-15")
    all_chart=all_chart.append(chart.shotchart())
    shots = shot_log("2014-15",row["PERSON_ID"])
    shots['PLAYER_ID']=row["PERSON_ID"]
    all_shots = all_shots.append(shots)
all_shots.to_csv("all_shots_2014.csv")
all_chart.to_csv("all_chart_2014.csv")

In [295]:
#Get shot chart and shot log for all players in 2013
#Don't run, already been saved
all_shots= pd.DataFrame()
all_chart = pd.DataFrame()
for index, row in season2013.iterrows():
    shots = shot_log("2013-14",row["PERSON_ID"])
    shots['PLAYER_ID']=row["PERSON_ID"]
    all_shots = all_shots.append(shots)
    chart = nbastats.ShotChart(row["PERSON_ID"],season="2013-14")
    all_chart=all_chart.append(chart.shotchart())
all_shots.to_csv("all_shots_2013.csv")
all_chart.to_csv("all_chart_2013.csv")

In [5]:
## Merging shot chart data with player tracking for 2014
## DO NOT RUN... takes a while
all_shots = pd.read_csv("all_shots_2013.csv")
all_chart = pd.read_csv("all_chart_2013.csv")
players=set(all_shots['PLAYER_ID'])
joined_shots=pd.DataFrame()
joined_shots_1=pd.DataFrame()
for player in players:
    shots = all_shots.loc[all_shots.PLAYER_ID==player].reset_index(drop=True)
    chart = all_chart.loc[all_chart.PLAYER_ID==player].reset_index(drop=True)
    split_time = shots.GAME_CLOCK.str.split(':')
    col_names=[u'CLOSEST_DEFENDER', u'CLOSEST_DEFENDER_PLAYER_ID', u'CLOSE_DEF_DIST', 
               u'DRIBBLES', u'FGM', u'FINAL_MARGIN', u'GAME_CLOCK', u'GAME_ID', u'LOCATION', 
               u'MATCHUP', u'PERIOD', u'PTS', u'PTS_TYPE', u'SHOT_CLOCK', u'SHOT_DIST', 
               u'SHOT_NUMBER', u'SHOT_RESULT', u'TOUCH_TIME', u'W',u'GRID_TYPE',u'GAME_EVENT_ID',
               u'PLAYER_ID', u'PLAYER_NAME', u'TEAM_ID',u'MINUTES_REMAINING', u'SECONDS_REMAINING', 
               u'EVENT_TYPE', u'ACTION_TYPE', u'SHOT_TYPE', u'SHOT_ZONE_BASIC', 
               u'SHOT_ZONE_AREA', u'SHOT_ZONE_RANGE', u'SHOT_DISTANCE', u'LOC_X', u'LOC_Y', 
               u'SHOT_ATTEMPTED_FLAG', u'SHOT_MADE_FLAG']
    time = pd.DataFrame(shots.GAME_CLOCK.str.split(':').tolist(), columns="MIN SECONDS".split())
    shots['MIN'] = time['MIN']
    shots['SECONDS'] = time['SECONDS']
    shots['TIME']=60*shots.MIN.astype(int)+shots.SECONDS.astype(int)
    chart['TIME']=60*chart.MINUTES_REMAINING+chart.SECONDS_REMAINING
    games = set(chart["GAME_ID"])
    for game in games:
        game_shots_chart=chart.loc[chart.GAME_ID==game].reset_index(drop=True)
        game_shots_log = shots.loc[shots.GAME_ID==game].reset_index(drop=True)
        if game_shots_chart.shape[0]==game_shots_log.shape[0]:
            huh = pd.concat([game_shots_chart,game_shots_log],axis=1)
            huh=huh[col_names]
            joined_shots=joined_shots.append(huh)
        else:
            quarters=set(game_shots_chart['PERIOD'])
            for quarter in quarters:
                same_period=game_shots_log.loc[(game_shots_log.PERIOD==quarter)].reset_index(drop=True)
                same_period_chart=game_shots_chart.loc[(game_shots_chart.PERIOD==quarter)].reset_index(drop=True)
                if (not same_period.empty) and (not same_period_chart.empty):
                    if same_period_chart.shape[0]==same_period.shape[0]:
                        huh = pd.concat([same_period_chart,same_period],axis=1)
                        huh=huh[col_names]
                        joined_shots=joined_shots.append(huh)
                    else:
                        for index,row in same_period_chart.iterrows():
                            same_period['DIF']=abs(same_period.TIME-row.TIME)
                            df=same_period.sort(['DIF'],ascending=True)
                            same_time=same_period.loc[[0]]
                            if same_time.DIF[0]<5:
                                a=same_period_chart.loc[[index]].reset_index(drop=True)
                                huh=pd.concat([a,same_time],axis=1)
                                huh=huh[col_names]
                                joined_shots_1=joined_shots_1.append(huh,ignore_index=True)
    print player
joined_shots

202779
200745
200746
200751
200752
200755
200756
200757
200758
200761
2399
200765
200768
200769
200770
200777
200779
200780
200782
200794
200797
2405
2406
200811
203112
200826
200839
203458
203459
101107
2199
2200
203460
2203
2207
2210
2215
2216
2222
2223
2224
2225
2229
202952
202962
202970
203471
203476
101108
203144
2365
203076
203077
203078
203079
203080
203081
203082
203083
203084
203085
203086
203087
203088
203089
203090
203091
203092
203093
203094
203095
203096
203098
203099
203100
203101
203102
203103
2400
203106
2403
203108
203109
203110
203111
2408
203113
203114
203115
203116
203117
203118
203120
203121
2419
203124
2422
203129
203130
2427
203133
2430
203135
203136
203138
203139
203141
203142
203143
2440
203145
203146
203148
2446
2449
203156
2457
203506
203183
203187
201141
201142
201143
201144
201145
201147
201148
201149
201150
201152
201155
201156
2501
201158
201160
201162
201163
201166
201167
201168
201171
201175
201177
203515
201188
201189
201196
2544
201202
2547
2548
2549


CLOSEST_DEFENDER  CLOSEST_DEFENDER_PLAYER_ID  CLOSE_DEF_DIST  DRIBBLES  FGM  FINAL_MARGIN GAME_CLOCK   GAME_ID   GAME_ID LOCATION                     MATCHUP  PERIOD  PERIOD  PTS  PTS_TYPE  SHOT_CLOCK  SHOT_DIST  SHOT_NUMBER SHOT_RESULT  TOUCH_TIME  W          GRID_TYPE  GAME_EVENT_ID  PLAYER_ID  PLAYER_ID    PLAYER_NAME     TEAM_ID  MINUTES_REMAINING  SECONDS_REMAINING   EVENT_TYPE             ACTION_TYPE       SHOT_TYPE        SHOT_ZONE_BASIC         SHOT_ZONE_AREA  \
0               Turner, Evan                      202323             1.9        17    0            10       8:09  21300165  21300165        A    NOV 20, 2013 - TOR @ PHI       2       2    0         2         9.8        7.7            1      missed        13.5  W  Shot Chart Detail            163     202779     202779  Dwight Buycks  1610612761                  8                  6  Missed Shot               Jump Shot  2PT Field Goal  In The Paint (Non-RA)              Center(C)   
1               Turner, Evan                      202323             3.2         0    1            10       9:10  21300165  21300165        A    NOV 20, 2013 - TOR @ PHI       4       4    3         3         8.1       23.5            2        made         1.4  W  Shot Chart Detail            449     202779     202779  Dwight Buycks  1610612761                  9                  7    Made Shot               Jump Shot  3PT Field Goal      Above the Break 3  Right Side Center(RC)   
2   Carter-Williams, Michael                      203487             3.5         0    0            10       8:36  21300165  21300165        A    NOV 20, 2013 - TOR @ PHI       4       4    0         3        11.0       25.7            3      missed         1.3  W  Shot Chart Detail            452     202779     202779  Dwight Buycks  1610612761                  8                 33  Missed Shot               Jump Shot  3PT Field Goal      Above the Break 3  Right Side Center(RC)   
3             Brown, Lorenzo                      203485             3.6        19    0            10       0:23  21300165  21300165        A    NOV 20, 2013 - TOR @ PHI       4       4    0         3         9.0       24.7            4      missed        14.5  W  Shot Chart Detail            575     202779     202779  Dwight Buycks  1610612761                  0                 21  Missed Shot               Jump Shot  3PT Field Goal      Above the Break 3              Center(C)   
0               Cole, Norris                      202708             3.7         0    1            -7      11:43  21300230  21300230        H  NOV 29, 2013 - TOR vs. MIA       2       2    3         3         NaN       25.2            1        made         1.0  L  Shot Chart Detail            134     202779     202779  Dwight Buycks  1610612761                 11                 41    Made Shot               Jump Shot  3PT Field Goal      Above the Break 3   Left Side Center(LC)   
1            Andersen, Chris                        2365             1.6        13    0            -7       9:36  21300230  21300230        H  NOV 29, 2013 - TOR vs. MIA       2       2    0         2         NaN        9.3            2      missed         9.3  L  Shot Chart Detail            146     202779     202779  Dwight Buycks  1610612761                  9                 34  Missed Shot               Jump Shot  2PT Field Goal              Mid-Range           Left Side(L)   
2               Cole, Norris                      202708             2.4         5    0            -7       8:21  21300230  21300230        H  NOV 29, 2013 - TOR vs. MIA       2       2    0         3         2.6       28.4            3      missed         4.5  L  Shot Chart Detail            159     202779     202779  Dwight Buycks  1610612761                  8                 18  Missed Shot               Jump Shot  3PT Field Goal      Above the Break 3   Left Side Center(LC)   
0            Jones, Terrence                      203093             3.4         6    0            -6      10:48  21

In [6]:
## Saving what we merged, checking the shape to see how many shots we didn't have
joined_shots.to_csv("joined_shots_2013.csv")
print joined_shots.shape,all_shots.shape,all_chart.shape

(200963, 40) (202689, 21) (204126, 22)


In [ ]:
##NEED TO DO THE SAME FOR 2013

In [345]:
joined_shots=pd.read_csv("joined_shots_2014.csv")
max(joined_shots.LOC_Y)
plt.scatter(joined_shots.LOC_X,joined_shots.LOC_Y)
plt.show()

In [10]:
all2013=pd.read_csv('joined_shots_2013.csv')

In [11]:
all2013['off_height']=0
all2013['def_height']=0

In [73]:
players_shot=list(set(all2013['PLAYER_ID']))
players_def=list(set(all2013['CLOSEST_DEFENDER_PLAYER_ID']))

In [71]:
at = all2013[all2013['PLAYER_ID']==2544].index.tolist()
all2013.loc[at]

Unnamed: 0    CLOSEST_DEFENDER  CLOSEST_DEFENDER_PLAYER_ID  CLOSE_DEF_DIST  DRIBBLES  FGM  FINAL_MARGIN GAME_CLOCK   GAME_ID  GAME_ID.1 LOCATION                     MATCHUP  PERIOD  PERIOD.1  PTS  PTS_TYPE  SHOT_CLOCK  SHOT_DIST  SHOT_NUMBER SHOT_RESULT  TOUCH_TIME  W          GRID_TYPE  GAME_EVENT_ID  PLAYER_ID  PLAYER_ID.1   PLAYER_NAME     TEAM_ID  MINUTES_REMAINING  SECONDS_REMAINING   EVENT_TYPE               ACTION_TYPE       SHOT_TYPE        SHOT_ZONE_BASIC  \
63215           0     Crawford, Jamal                        2037             4.1         2    1             4       9:27  21300736   21300736        A    FEB 05, 2014 - MIA @ LAC       1         1    2         2        13.1        1.5            1        made         1.4  W  Shot Chart Detail             19       2544         2544  LeBron James  1610612748                  9                 26    Made Shot         Driving Dunk Shot  2PT Field Goal        Restricted Area   
63216           1     Jordan, DeAndre                      201599             5.5         0    1             4       7:38  21300736   21300736        A    FEB 05, 2014 - MIA @ LAC       1         1    2         2        21.7        5.8            2        made         0.4  W  Shot Chart Detail             31       2544         2544  LeBron James  1610612748                  7                 37    Made Shot         Driving Dunk Shot  2PT Field Goal        Restricted Area   
63217           2        Barnes, Matt                        2440             1.4         3    1             4       4:57  21300736   21300736        A    FEB 05, 2014 - MIA @ LAC       1         1    2         2        19.4        4.5            3        made         3.5  W  Shot Chart Detail             53       2544         2544  LeBron James  1610612748                  4                 55    Made Shot        Driving Layup Shot  2PT Field Goal        Restricted Area   
63218           3     Jordan, DeAndre                      201599             6.2        11    0             4       1:23  21300736   21300736        A    FEB 05, 2014 - MIA @ LAC       1         1    0         3        14.1       24.8            4      missed         9.6  W  Shot Chart Detail             82       2544         2544  LeBron James  1610612748                  1                 20  Missed Shot                 Jump Shot  3PT Field Goal      Above the Break 3   
63219           4      Griffin, Blake                      201933             4.9         2    0             4       0:53  21300736   21300736        A    FEB 05, 2014 - MIA @ LAC       1         1    0         2         3.0       21.7            5      missed         5.5  W  Shot Chart Detail             85       2544         2544  LeBron James  1610612748                  0                 50  Missed Shot                 Jump Shot  2PT Field Goal              Mid-Range   
63220           5      Griffin, Blake                      201933             1.0         5    1             4       6:07  21300736   21300736        A    FEB 05, 2014 - MIA @ LAC       2         2    2         2         7.0        3.7            6        made         5.1  W  Shot Chart Detail            188       2544         2544  LeBron James  1610612748                  6                  5    Made Shot        Driving Layup Shot  2PT Field Goal        Restricted Area   
63221           6        Barnes, Matt                        2440             1.0         6    0             4       4:50  21300736   21300736        A    FEB 05, 2014 - MIA @ LAC       2         2    0         2        18.5        6.8            7      missed         5.1  W  Shot Chart Detail            196       2544         2544  LeBron James  1610612748                  4                 55  Missed Shot      Turnaround Jump Shot  2PT Field Goal        Restricted Area   
63222           7        Barnes, Matt                        2440             2.0         0    1             4       4:48  21300736   21300736        A    FEB 05, 2014 - MIA @

In [75]:
for player in players_shot:
    at = all2013[all2013['PLAYER_ID']==player].index.tolist()
    hi = players2013.loc[players2013.PERSON_ID==player]['HEIGHT']
    all2013['off_height'][at]=hi.iloc[0]
for player in players_def:
    at = all2013[all2013['CLOSEST_DEFENDER_PLAYER_ID']==player].index.tolist()
    hi = players2013.loc[players2013.PERSON_ID==player]['HEIGHT']
    all2013['def_height'][at]=hi.iloc[0]

/Users/harrisonchase/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/harrisonchase/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [36]:
all2013[all2013['PLAYER_ID']==2544].index.tolist()
all2013['off_height'][[1,2]]

1    0
2    0
Name: off_height, dtype: int64

In [77]:
plt.scatter(all2013['SHOT_DIST'],all2013['SHOT_DISTANCE'],alpha=.2)
plt.show()

/Users/harrisonchase/anaconda/lib/python2.7/site-packages/matplotlib/collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):


In [15]:
int(all2013.iloc[0]['PLAYER_ID'])
players2013.loc[players2013.PERSON_ID==all2013.iloc[0]['PLAYER_ID']]['PERSON_ID']

515    202779
Name: PERSON_ID, dtype: int64

In [ ]:
for index,row in all2013.iterrows():
    if index % 1000==0:
        print index
    all2013.iloc[index]['off_height']=players2013.loc[players2013.PERSON_ID==row['PLAYER_ID']]['HEIGHT']

In [79]:
### Merge play by play and shots
pbp2013=pd.read_csv('playbyplay/pbp_with_lineups_2013.csv')

/Users/harrisonchase/anaconda/lib/python2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (43) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [94]:
sort=pbp2013.sort(['GAME_ID','EVENTNUM'])

In [103]:
huh = sort.drop_duplicates(['GAME_ID','EVENTNUM'])

In [115]:
len(["nan"]+list(huh.EVENTMSGTYPE))

540784

In [116]:
huh['ACTION_BEFORE']=["nan"]+list(huh.EVENTMSGTYPE)[:-1]
huh.head()

/Users/harrisonchase/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


Unnamed: 0  AWAY_PLAYER1_ID  AWAY_PLAYER2_ID  AWAY_PLAYER3_ID  AWAY_PLAYER4_ID  AWAY_PLAYER5_ID  AWAY_TEAM_ID  EVENTMSGACTIONTYPE  EVENTMSGTYPE  EVENTNUM   GAME_ID                             HOMEDESCRIPTION  HOME_PLAYER1_ID  HOME_PLAYER2_ID  HOME_PLAYER3_ID  HOME_PLAYER4_ID  HOME_PLAYER5_ID  HOME_TEAM_ID  NEUTRALDESCRIPTION PCTIMESTRING  PERIOD  PERSON1TYPE  PERSON2TYPE  PERSON3TYPE  PLAYER1_ID    PLAYER1_NAME PLAYER1_TEAM_ABBREVIATION PLAYER1_TEAM_CITY  PLAYER1_TEAM_ID  \
0                0             2749           101131           201167           202696           203090    1610612753                   0            12         0  21300001                                         NaN             2561           201579           201588           202331           202362    1610612754                 NaN        12:00       1            0            0            0           0             NaN                       NaN               NaN              NaN   
1                1             2749           101131           201167           202696           203090    1610612753                   0            10         1  21300001  Jump Ball Hibbert vs. Vucevic: Tip to West             2561           201579           201588           202331           202362    1610612754                 NaN        12:00       1            4            5            4      201579     Roy Hibbert                       IND           Indiana       1610612754   
111112           2             2749           101131           201167           202696           203090    1610612753                   1             2         2  21300001                      MISS West 6' Jump Shot             2561           201579           201588           202331           202362    1610612754                 NaN        11:39       1            4            0            0        2561      David West                       IND           Indiana       1610612754   
222223           3             2749           101131           201167           202696           203090    1610612753                   0             4         3  21300001               Hibbert REBOUND (Off:1 Def:0)             2561           201579           201588           202331           202362    1610612754                 NaN        11:39       1            4            0            0      201579     Roy Hibbert                       IND           Indiana       1610612754   
333334           4             2749           101131           201167           202696           203090    1610612753                   3             6         4  21300001                                         NaN             2561           201579           201588           202331           202362    1610612754                 NaN        11:39       1            5            4            1      202696  Nikola Vucevic                       ORL           Orlando       1610612753   

       PLAYER1_TEAM_NICKNAME  PLAYER2_ID    PLAYER2_NAME PLAYER2_TEAM_ABBREVIATION PLAYER2_TEAM_CITY  PLAYER2_TEAM_ID PLAYER2_TEAM_NICKNAME  PLAYER3_ID PLAYER3_NAME PLAYER3_TEAM_ABBREVIATION PLAYER3_TEAM_CITY  PLAYER3_TEAM_ID PLAYER3_TEAM_NICKNAME SCORE SCOREMARGIN        VISITORDESCRIPTION WCTIMESTRING ACTION_BEFORE  
0                        NaN           0             NaN                       NaN               NaN              NaN                   NaN           0          NaN                       NaN               NaN              NaN                   NaN   NaN         NaN                       NaN      7:12 PM           nan  
1                     Pacers      202696  Nikola Vucevic                       ORL           Orlando       1610612753                 Magic        2561   David West                       IND           Indiana       1610612754                Pacers   NaN         NaN                       NaN      7:12 PM            12  
111112                Pacers           0             NaN                       NaN               NaN              NaN     

In [87]:
all2013['EVENTNUM']=all2013['GAME_EVENT_ID']

In [117]:
try1=huh.merge(all2013,on=['GAME_ID','EVENTNUM'])

In [118]:
try1.head()

Unnamed: 0_x  AWAY_PLAYER1_ID  AWAY_PLAYER2_ID  AWAY_PLAYER3_ID  AWAY_PLAYER4_ID  AWAY_PLAYER5_ID  AWAY_TEAM_ID  EVENTMSGACTIONTYPE  EVENTMSGTYPE  EVENTNUM   GAME_ID                                    HOMEDESCRIPTION  HOME_PLAYER1_ID  HOME_PLAYER2_ID  HOME_PLAYER3_ID  HOME_PLAYER4_ID  HOME_PLAYER5_ID  HOME_TEAM_ID  NEUTRALDESCRIPTION PCTIMESTRING  PERIOD_x  PERSON1TYPE  PERSON2TYPE  PERSON3TYPE  PLAYER1_ID      PLAYER1_NAME PLAYER1_TEAM_ABBREVIATION PLAYER1_TEAM_CITY  PLAYER1_TEAM_ID  \
0             2             2749           101131           201167           202696           203090    1610612753                   1             2         2  21300001                             MISS West 6' Jump Shot             2561           201579           201588           202331           202362    1610612754                 NaN        11:39         1            4            0            0        2561        David West                       IND           Indiana       1610612754   
1           470             2749           101131           201167           202696           203090    1610612753                  79             1         5  21300001                George 19' Pullup Jump Shot (2 PTS)             2561           201579           201588           202331           202362    1610612754                 NaN        11:30         1            4            0            0      202331       Paul George                       IND           Indiana       1610612754   
2           471             2749           101131           201167           202696           203090    1610612753                   1             2         6  21300001                                                NaN             2561           201579           201588           202331           202362    1610612754                 NaN        11:15         1            5            0            0      201167     Arron Afflalo                       ORL           Orlando       1610612753   
3           473             2749           101131           201167           202696           203090    1610612753                   5             2         8  21300001                                 MISS West 3' Layup             2561           201579           201588           202331           202362    1610612754                 NaN        11:03         1            4            0            5        2561        David West                       IND           Indiana       1610612754   
4            11             2749           101131           201167           202696           203090    1610612753                   1             1        11  21300001  Stephenson 26' 3PT Jump Shot (3 PTS) (West 1 AST)             2561           201579           201588           202331           202362    1610612754                 NaN        10:51         1            4            4            0      202362  Lance Stephenson                       IND           Indiana       1610612754   

  PLAYER1_TEAM_NICKNAME  PLAYER2_ID PLAYER2_NAME PLAYER2_TEAM_ABBREVIATION PLAYER2_TEAM_CITY  PLAYER2_TEAM_ID PLAYER2_TEAM_NICKNAME  PLAYER3_ID   PLAYER3_NAME PLAYER3_TEAM_ABBREVIATION PLAYER3_TEAM_CITY  PLAYER3_TEAM_ID PLAYER3_TEAM_NICKNAME  SCORE SCOREMARGIN              VISITORDESCRIPTION WCTIMESTRING ACTION_BEFORE  Unnamed: 0_y   CLOSEST_DEFENDER  CLOSEST_DEFENDER_PLAYER_ID  CLOSE_DEF_DIST  DRIBBLES  FGM  FINAL_MARGIN GAME_CLOCK  GAME_ID.1 LOCATION                     MATCHUP  PERIOD_y  \
0                Pacers           0          NaN                       NaN               NaN              NaN                   NaN           0            NaN                       NaN               NaN              NaN                   NaN    NaN         NaN                             NaN      7:12 PM            10             0     Maxiell, Jason                      101131             1.6         2    0            10      11:41   21300001        H  OCT 29, 2013 - IND vs. ORL         1   
1                Pacers           0 

In [121]:
try1.to_csv('merged_shots_13.csv')

In [132]:
all2014=pd.read_csv('joined_shots_2014.csv')
all2014['off_height']=0
all2014['def_height']=0
players_shot=list(set(all2014['PLAYER_ID']))
players_def=list(set(all2014['CLOSEST_DEFENDER_PLAYER_ID']))
for player in players_shot:
    at = all2014[all2014['PLAYER_ID']==player].index.tolist()
    hi = players2014.loc[players2014.PERSON_ID==player]['HEIGHT']
    all2014['off_height'][at]=hi.iloc[0]
for player in players_def:
    at = all2014[all2014['CLOSEST_DEFENDER_PLAYER_ID']==player].index.tolist()
    hi = players2014.loc[players2014.PERSON_ID==player]['HEIGHT']
    if len(hi)>0:
        all2014['def_height'][at]=hi.iloc[0]

/Users/harrisonchase/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/harrisonchase/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


done
done


/Users/harrisonchase/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


KeyError: 'EVENTNUM'

In [133]:
pbp2014=pd.read_csv('playbyplay/pbp_with_lineups_2014.csv')
sort=pbp2014.sort(['GAME_ID','EVENTNUM'])
all2014['EVENTNUM']=all2014['GAME_EVENT_ID']
huh = sort.drop_duplicates(['GAME_ID','EVENTNUM'])
huh['ACTION_BEFORE']=["nan"]+list(huh.EVENTMSGTYPE)[:-1]
try1=huh.merge(all2014,on=['GAME_ID','EVENTNUM'])
try1.to_csv('merged_shots_14.csv')

/Users/harrisonchase/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
